In [41]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer, AerSimulator
from qiskit.quantum_info import Statevector, state_fidelity
from qiskit_aer.noise import NoiseModel, amplitude_damping_error, depolarizing_error, phase_damping_error, pauli_error, kraus_error
from qiskit_ibm_runtime.fake_provider import FakeManhattanV2

import time
import tracemalloc
import csv


In [42]:
backend = FakeManhattanV2()


In [43]:

# Define Quantum Error Correction Code Circuits

def shor_code_circuit(include_measure=True):
    qc = QuantumCircuit(9)
    
    # Encode logical |0> state (bit-flip encoding)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)
    
    qc.cx(3, 4)
    qc.cx(3, 5)
    
    qc.cx(6, 7)
    qc.cx(6, 8)
    
    # Phase-flip encoding (using CNOTs and H gates)
    for i in [0, 3, 6]:
        qc.h(i)
        qc.cx(i, i+1)
        qc.cx(i, i+2)
        qc.h(i)
    
    # Bit-flip protection
    qc.cx(0, 3)
    qc.cx(0, 6)
    qc.cx(1, 4)
    qc.cx(1, 7)
    qc.cx(2, 5)
    qc.cx(2, 8)

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc

def steane_code_circuit(include_measure=True):
    qc = QuantumCircuit(7)

    # Encode logical |0> state
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)
    
    qc.cx(0, 3)
    qc.cx(1, 4)
    qc.cx(2, 5)
    
    qc.h(0)
    qc.h(1)
    qc.h(2)
    
    qc.cx(0, 6)

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc

def surface_code_circuit(include_measure=True):
    qc = QuantumCircuit(13)

    # Simplified example of a surface code (not a full implementation)
    # Initialize logical |0> state
    qc.h([0, 1, 2])
    
    # Create stabilizers
    for i in range(1, 7):
        qc.cx(0, i)
        qc.cz(0, i+6)

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc



In [44]:
noise_model = NoiseModel.from_backend(backend)


In [45]:



# Noise Models
def create_depolarizing_noise_model(noise_level):
    noise_model = NoiseModel()
    depol_error_1q = depolarizing_error(noise_level, 1)
    depol_error_2q = depolarizing_error(noise_level, 2)
    noise_model.add_all_qubit_quantum_error(depol_error_1q, ["u3", "u2", "u1"])
    noise_model.add_all_qubit_quantum_error(depol_error_2q, ["cx"])
    return noise_model

def create_amplitude_damping_noise_model(noise_level):
    noise_model = NoiseModel()
    amp_damp_error_1q = amplitude_damping_error(noise_level)
    noise_model.add_all_qubit_quantum_error(amp_damp_error_1q, ["u3"])
    return noise_model

def create_phase_damping_noise_model(noise_level):
    noise_model = NoiseModel()
    phase_damp_error_1q = phase_damping_error(noise_level)
    noise_model.add_all_qubit_quantum_error(phase_damp_error_1q, ["u1"])
    return noise_model

def create_bit_flip_noise_model(noise_level):
    noise_model = NoiseModel()
    bit_flip_error = pauli_error([('X', noise_level), ('I', 1 - noise_level)])
    noise_model.add_all_qubit_quantum_error(bit_flip_error, ["x"])
    return noise_model

def create_phase_flip_noise_model(noise_level):
    noise_model = NoiseModel()
    phase_flip_error = pauli_error([('Z', noise_level), ('I', 1 - noise_level)])
    noise_model.add_all_qubit_quantum_error(phase_flip_error, ["z"])
    return noise_model

def create_bit_phase_flip_noise_model(noise_level):
    noise_model = NoiseModel()
    bit_phase_flip_error = pauli_error([('Y', noise_level), ('I', 1 - noise_level)])
    noise_model.add_all_qubit_quantum_error(bit_phase_flip_error, ["y"])
    return noise_model

def create_kraus_error_model(noise_level):
    noise_model = NoiseModel()
    K0 = np.array([[1, 0], [0, np.sqrt(1 - noise_level)]])
    K1 = np.array([[0, np.sqrt(noise_level)], [0, 0]])
    kraus_ops = [K0, K1]
    error = kraus_error(kraus_ops)
    noise_model.add_all_qubit_quantum_error(error, ["h", "u3", "x", "z"])
    return noise_model

In [46]:

from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.passes import SetLayout, FullAncillaAllocation, EnlargeWithAncilla, ApplyLayout

# Initialize the FakeMelbourne Backend
backend = FakeMelbourne()

# Ensure that backend properties are properly initialized
if backend.configuration().coupling_map is None or backend.configuration().basis_gates is None:
    raise ValueError("The backend's coupling map or basis gates are not properly initialized.")

# Define Quantum Error Correction Code Circuits
def shor_code_circuit(include_measure=True):
    qc = QuantumCircuit(9)
    
    # Encode logical |0> state (bit-flip encoding)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)
    
    qc.cx(3, 4)
    qc.cx(3, 5)
    
    qc.cx(6, 7)
    qc.cx(6, 8)
    
    # Phase-flip encoding (using CNOTs and H gates)
    for i in [0, 3, 6]:
        qc.h(i)
        qc.cx(i, i+1)
        qc.cx(i, i+2)
        qc.h(i)
    
    # Bit-flip protection
    qc.cx(0, 3)
    qc.cx(0, 6)
    qc.cx(1, 4)
    qc.cx(1, 7)
    qc.cx(2, 5)
    qc.cx(2, 8)

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc

def steane_code_circuit(include_measure=True):
    qc = QuantumCircuit(7)

    # Encode logical |0> state
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)
    
    qc.cx(0, 3)
    qc.cx(1, 4)
    qc.cx(2, 5)
    
    qc.h(0)
    qc.h(1)
    qc.h(2)
    
    qc.cx(0, 6)

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc

def surface_code_circuit(include_measure=True):
    qc = QuantumCircuit(13)

    # Simplified example of a surface code (not a full implementation)
    # Initialize logical |0> state
    qc.h([0, 1, 2])
    
    # Create stabilizers
    for i in range(1, 7):
        qc.cx(0, i)
        qc.cz(0, i+6)

    # Optionally include measurement
    if include_measure:
        qc.measure_all()
    
    return qc

# Validate Circuit without noise
def validate_circuit_ideal(code_name):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit()
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit()
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit()
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Transpile the circuit for the backend with explicit layout and routing method
    try:
        qc_transpiled = transpile(qc_circuit, backend, optimization_level=0, routing_method='basic', initial_layout=None)
        # Run the circuit and retrieve the measurement result
        result = backend.run(qc_transpiled).result()
        counts = result.get_counts(qc_transpiled)
        print(f"Measurement counts for {code_name} code:\n", counts)
    except Exception as e:
        print(f"Error in transpiling or running the circuit for {code_name} code: {str(e)}")

# Validate each QEC circuit without noise
validate_circuit_ideal('shor')
validate_circuit_ideal('steane')
validate_circuit_ideal('surface')


Measurement counts for shor code:
 {'011010101': 1, '010000001': 1, '010010111': 1, '110100011': 1, '000010100': 1, '000101100': 1, '100110010': 1, '001111110': 3, '111100100': 2, '110001011': 1, '001110010': 2, '000010011': 1, '100100000': 1, '011010100': 1, '100010000': 1, '100000001': 1, '101001011': 3, '000111110': 1, '100101101': 1, '010001101': 1, '100011001': 1, '010110010': 3, '111100101': 1, '010000110': 2, '110010010': 2, '011000011': 1, '100001011': 1, '000000100': 1, '101111100': 1, '110101110': 1, '101111101': 1, '111101110': 1, '101010000': 1, '100011010': 1, '110100110': 2, '101101110': 1, '010111100': 3, '100010101': 1, '110011001': 4, '100111001': 1, '010110110': 1, '000101000': 1, '000010010': 3, '000010101': 2, '000111111': 1, '100111011': 1, '101000010': 2, '001111111': 1, '011000001': 6, '010010100': 2, '001001100': 4, '000011110': 3, '111110100': 2, '111111111': 1, '110100111': 1, '010110011': 1, '000101110': 1, '011100000': 2, '011110110': 3, '011000111': 1, '001

In [50]:


def calculate_error_rate(ideal_counts, noisy_counts, shots):
    """Calculate the error rate based on measurement counts."""
    ideal_total = sum(ideal_counts.values())
    correct_counts = sum(noisy_counts.get(bitstring, 0) for bitstring in ideal_counts)
    error_rate = 1 - (correct_counts / shots)
    return error_rate

def simulate_with_qec(noise_model, backend, code_name, shots=1024):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit()
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit()
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit()
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Transpile the circuit for the backend
    qc_transpiled = transpile(qc_circuit, backend, optimization_level=0)
    
    try:
        # Run the circuit on the backend with noise
        job = backend.run(qc_transpiled, noise_model=noise_model, shots=shots)
        result_noisy = job.result()
        noisy_counts = result_noisy.get_counts(qc_transpiled)
        
        # Define the ideal output as the all-zero bitstring
        ideal_counts = {'0' * qc_circuit.num_qubits: shots}

        # Calculate error rate based on counts
        error_rate = calculate_error_rate(ideal_counts, noisy_counts, shots)

        # Calculate fidelity based on counts (simplified fidelity estimation)
        fidelity = 1 - error_rate
        
        return fidelity, error_rate
    except Exception as e:
        print(f"Error in simulation for {code_name} code: {str(e)}")
        return None, None

def simulate_with_all_noise_models(noise_level, backend):
    noise_models = {
        "Depolarizing": create_depolarizing_noise_model(noise_level),
        "Amplitude Damping": create_amplitude_damping_noise_model(noise_level),
        "Phase Damping": create_phase_damping_noise_model(noise_level),
        "Bit Flip": create_bit_flip_noise_model(noise_level),
        "Phase Flip": create_phase_flip_noise_model(noise_level),
        "Bit Phase Flip": create_bit_phase_flip_noise_model(noise_level),
        "Kraus": create_kraus_error_model(noise_level)
    }
    
    codes = ["shor", "steane", "surface"]
    results = []
    
    for code_name in codes:
        for noise_name, noise_model in noise_models.items():
            fidelity, error_rate = simulate_with_qec(noise_model, backend, code_name)
            results.append({
                "Code": code_name,
                "Noise Model": noise_name,
                "Fidelity": fidelity,
                "Error Rate": error_rate
            })
            print(f"Code: {code_name}, Noise Model: {noise_name}, Fidelity: {fidelity}, Error Rate: {error_rate}")
    
    return results

# Example: Simulate all codes with all noise models
noise_level = 0.01  # You can adjust this level as needed
results = simulate_with_all_noise_models(noise_level, backend)


Code: shor, Noise Model: Depolarizing, Fidelity: 0.001953125, Error Rate: 0.998046875
Code: shor, Noise Model: Amplitude Damping, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Phase Damping, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Bit Flip, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Phase Flip, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Bit Phase Flip, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Kraus, Fidelity: 0.0, Error Rate: 1.0
Code: steane, Noise Model: Depolarizing, Fidelity: 0.05078125, Error Rate: 0.94921875
Code: steane, Noise Model: Amplitude Damping, Fidelity: 0.0615234375, Error Rate: 0.9384765625
Code: steane, Noise Model: Phase Damping, Fidelity: 0.0546875, Error Rate: 0.9453125
Code: steane, Noise Model: Bit Flip, Fidelity: 0.0869140625, Error Rate: 0.9130859375
Code: steane, Noise Model: Phase Flip, Fidelity: 0.0712890625, Error Rate: 0.9287109375
Code: steane, Noise Model: Bit Phase Flip, Fidelity: 0.0634

In [52]:
def simulate_no_correction(noise_model, backend):
    # Simple circuit without QEC
    qc = QuantumCircuit(1, 1)
    qc.h(0)
    
    # Transpile for the backend (without measurement for statevector)
    qc_transpiled_no_measure = transpile(qc, backend)
    
    # Evolve state vector without measurement
    state_noisy = Statevector.from_int(0, 2)
    state_noisy = state_noisy.evolve(qc_transpiled_no_measure)
    fidelity = state_fidelity(state_noisy, Statevector([1, 0]))  # Ideal state
    
    # Add measurement and transpile again for getting counts
    qc.measure(0, 0)
    qc_transpiled_with_measure = transpile(qc, backend)
    
    # Run the simulation with noise to get the error rate
    result = backend.run(qc_transpiled_with_measure, noise_model=noise_model, shots=1024).result()
    counts = result.get_counts()
    error_rate = 1 - (counts.get('0', 0) / 1024)
    
    return error_rate, fidelity

# Example: Simulate error rates and fidelity before applying QEC codes
noise_level = 0.05  # Adjust noise level as needed
results = simulate_with_all_noise_models(noise_level, backend)

#print(f"Error Rate without QEC: {error_rate_no_correction}")
#print(f"Fidelity without QEC: {fidelity_no_correction}")


Code: shor, Noise Model: Depolarizing, Fidelity: 0.0068359375, Error Rate: 0.9931640625
Code: shor, Noise Model: Amplitude Damping, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Phase Damping, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Bit Flip, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Phase Flip, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Bit Phase Flip, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Kraus, Fidelity: 0.0, Error Rate: 1.0
Code: steane, Noise Model: Depolarizing, Fidelity: 0.0224609375, Error Rate: 0.9775390625
Code: steane, Noise Model: Amplitude Damping, Fidelity: 0.064453125, Error Rate: 0.935546875
Code: steane, Noise Model: Phase Damping, Fidelity: 0.0634765625, Error Rate: 0.9365234375
Code: steane, Noise Model: Bit Flip, Fidelity: 0.05859375, Error Rate: 0.94140625
Code: steane, Noise Model: Phase Flip, Fidelity: 0.0712890625, Error Rate: 0.9287109375
Code: steane, Noise Model: Bit Phase Flip, Fidelity: 

In [53]:
def test_with_gradual_noise(backend, code_name, initial_noise_level=0.001, step=0.001, max_noise_level=0.01):
    noise_level = initial_noise_level
    while noise_level <= max_noise_level:
        noise_models = {
            "Depolarizing": create_depolarizing_noise_model(noise_level),
            "Amplitude Damping": create_amplitude_damping_noise_model(noise_level),
            "Phase Damping": create_phase_damping_noise_model(noise_level),
            "Bit Flip": create_bit_flip_noise_model(noise_level),
            "Phase Flip": create_phase_flip_noise_model(noise_level),
            "Bit Phase Flip": create_bit_phase_flip_noise_model(noise_level),
            "Kraus": create_kraus_error_model(noise_level)
        }
        
        for noise_name, noise_model in noise_models.items():
            try:
                fidelity = simulate_with_qec(noise_model, backend, code_name)
                print(f"Noise Level: {noise_level}, Code: {code_name}, Noise Model: {noise_name}, Fidelity: {fidelity}")
            except Exception as e:
                print(f"Error in simulation for {code_name} with {noise_name}: {str(e)}")

        noise_level += step

# Gradually introduce noise for Shor code
test_with_gradual_noise(backend, 'shor')
test_with_gradual_noise(backend, 'steane')
test_with_gradual_noise(backend, 'surface')


Noise Level: 0.001, Code: shor, Noise Model: Depolarizing, Fidelity: (0.0, 1.0)
Noise Level: 0.001, Code: shor, Noise Model: Amplitude Damping, Fidelity: (0.0, 1.0)
Noise Level: 0.001, Code: shor, Noise Model: Phase Damping, Fidelity: (0.0, 1.0)
Noise Level: 0.001, Code: shor, Noise Model: Bit Flip, Fidelity: (0.0, 1.0)
Noise Level: 0.001, Code: shor, Noise Model: Phase Flip, Fidelity: (0.0, 1.0)
Noise Level: 0.001, Code: shor, Noise Model: Bit Phase Flip, Fidelity: (0.0, 1.0)
Noise Level: 0.001, Code: shor, Noise Model: Kraus, Fidelity: (0.0, 1.0)
Noise Level: 0.002, Code: shor, Noise Model: Depolarizing, Fidelity: (0.0009765625, 0.9990234375)
Noise Level: 0.002, Code: shor, Noise Model: Amplitude Damping, Fidelity: (0.0, 1.0)
Noise Level: 0.002, Code: shor, Noise Model: Phase Damping, Fidelity: (0.0, 1.0)
Noise Level: 0.002, Code: shor, Noise Model: Bit Flip, Fidelity: (0.0, 1.0)
Noise Level: 0.002, Code: shor, Noise Model: Phase Flip, Fidelity: (0.0, 1.0)
Noise Level: 0.002, Code: s

In [55]:
def analyze_circuit_depth(code_name):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit(include_measure=False)
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit(include_measure=False)
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit(include_measure=False)
    else:
        raise ValueError(f"Unknown code name: {code_name}")

    # Transpile and analyze depth with explicit parameters
    qc_transpiled = transpile(
        qc_circuit, 
        backend, 
        optimization_level=0,
        coupling_map=backend.configuration().coupling_map,
        basis_gates=backend.configuration().basis_gates
    )
    depth = qc_transpiled.depth()
    print(f"Circuit depth for {code_name} code: {depth}")

# Analyze circuit depth
analyze_circuit_depth('shor')
analyze_circuit_depth('steane')
analyze_circuit_depth('surface')


Circuit depth for shor code: 62
Circuit depth for steane code: 37
Circuit depth for surface code: 90


In [77]:


def simulate_code_threshold(noise_levels, code_name):
    thresholds = []
    fidelities = []
    results = []

    backend = FakeManhattanV2()

    for noise_level in noise_levels:
        noise_models = {
            "Depolarizing": create_depolarizing_noise_model(noise_level),
            "Amplitude Damping": create_amplitude_damping_noise_model(noise_level),
            "Phase Damping": create_phase_damping_noise_model(noise_level),
            "Bit Flip": create_bit_flip_noise_model(noise_level),
            "Phase Flip": create_phase_flip_noise_model(noise_level),
            "Bit Phase Flip": create_bit_phase_flip_noise_model(noise_level),
            "Kraus": create_kraus_error_model(noise_level)
        }

        for noise_name, noise_model in noise_models.items():
            try:
                if code_name == 'shor':
                    qc_circuit = shor_code_circuit(include_measure=True)
                elif code_name == 'steane':
                    qc_circuit = steane_code_circuit(include_measure=True)
                elif code_name == 'surface':
                    qc_circuit = surface_code_circuit(include_measure=True)
                else:
                    raise ValueError(f"Unknown code name: {code_name}")

                qc_transpiled = transpile(qc_circuit, backend)
                job_noisy = backend.run(qc_transpiled, noise_model=noise_model, shots=1024)
                result_noisy = job_noisy.result(timeout=600)

                counts_noisy = result_noisy.get_counts(qc_transpiled)
                if counts_noisy is None:
                    continue

                ideal_counts = {'0' * qc_circuit.num_clbits: 1024}
                correct_counts = sum(counts_noisy.get(bitstring, 0) for bitstring in ideal_counts)
                error_rate = 1 - (correct_counts / 1024)

                # Handling pure states: Converting to statevector and calculating fidelity
                qc_transpiled_no_measure = qc_circuit.remove_final_measurements(inplace=False)
                ideal_state = Statevector.from_label('0' * qc_circuit.num_qubits)
                noisy_state = Statevector.from_instruction(qc_transpiled_no_measure)
                fidelity = state_fidelity(noisy_state, ideal_state)

                thresholds.append(error_rate)
                fidelities.append(fidelity)
                results.append({
                    'Code': code_name,
                    'Noise_Model': noise_name,
                    'Noise_Level': noise_level,
                    'Error_Rate': error_rate,
                    'Fidelity': fidelity
                })

                print(f"Simulation complete for {code_name} with {noise_name} at noise level {noise_level}:")
                print(f"Error Rate: {error_rate}")
                print(f"Fidelity: {fidelity}")

                time.sleep(10)

            except Exception as e:
                print(f"Error in simulation for {code_name} with {noise_name}: {str(e)}")
                continue

    return thresholds, fidelities, results

# Example usage:
noise_levels = [0.01, 0.02, 0.03]
shor_thresholds, shor_fidelities, shor_results = simulate_code_threshold(noise_levels, 'shor')
steane_thresholds, steane_fidelities, steane_results = simulate_code_threshold(noise_levels, 'steane')
surface_thresholds, surface_fidelities, surface_results = simulate_code_threshold(noise_levels, 'surface')


Simulation complete for shor with Depolarizing at noise level 0.01:
Error Rate: 0.998046875
Fidelity: 0.0
Simulation complete for shor with Amplitude Damping at noise level 0.01:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Phase Damping at noise level 0.01:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Bit Flip at noise level 0.01:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Phase Flip at noise level 0.01:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Bit Phase Flip at noise level 0.01:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Kraus at noise level 0.01:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Depolarizing at noise level 0.02:
Error Rate: 0.9990234375
Fidelity: 0.0
Simulation complete for shor with Amplitude Damping at noise level 0.02:
Error Rate: 1.0
Fidelity: 0.0
Simulation complete for shor with Phase Damping at noise level 0.02:
Error Rate: 1.0
Fidelity: 0.0
Si

In [80]:
# Print the shapes and contents of the arrays
print("shor_thresholds shape:", shor_thresholds.shape, "content:", shor_thresholds)
print("steane_thresholds shape:", steane_thresholds.shape, "content:", steane_thresholds)
print("surface_thresholds shape:", surface_thresholds.shape, "content:", surface_thresholds)

print("shor_fidelities shape:", shor_fidelities.shape, "content:", shor_fidelities)
print("steane_fidelities shape:", steane_fidelities.shape, "content:", steane_fidelities)
print("surface_fidelities shape:", surface_fidelities.shape, "content:", surface_fidelities)


shor_thresholds shape: (21,) content: [0.99804688 1.         1.         1.         1.         1.
 1.         0.99902344 1.         1.         1.         1.
 1.         1.         0.99902344 1.         1.         1.
 1.         1.         1.        ]
steane_thresholds shape: (21,) content: [0.93457031 0.92480469 0.93359375 0.92871094 0.9453125  0.92382812
 0.94238281 0.9375     0.93652344 0.93066406 0.94335938 0.95117188
 0.94628906 0.93261719 0.94921875 0.921875   0.93261719 0.93847656
 0.93652344 0.94140625 0.93457031]
surface_thresholds shape: (21,) content: [0.88085938 0.88671875 0.88183594 0.87597656 0.85839844 0.86621094
 0.88964844 0.92773438 0.87792969 0.89257812 0.88183594 0.88085938
 0.86621094 0.87304688 0.9453125  0.88085938 0.86621094 0.86914062
 0.85351562 0.88574219 0.87597656]
shor_fidelities shape: (21,) content: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
steane_fidelities shape: (21,) content: [0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.06

In [ ]:

print("Surface code simulation result:", surface_result)
surface_thresholds.append(surface_error_rate)
surface_fidelities.append(surface_fidelity)


In [81]:
import numpy as np
import matplotlib.pyplot as plt

# Convert lists to numpy arrays
shor_thresholds = np.array(shor_thresholds)
steane_thresholds = np.array(steane_thresholds)
surface_thresholds = np.array(surface_thresholds)

shor_fidelities = np.array(shor_fidelities)
steane_fidelities = np.array(steane_fidelities)
surface_fidelities = np.array(surface_fidelities)

# Check the dimensions and compute means only if valid
if shor_thresholds.ndim == 2 and shor_thresholds.shape[1] == len(noise_levels):
    shor_mean_thresholds = np.mean(shor_thresholds, axis=0)
else:
    print("Issue with shor_thresholds dimensions")
    shor_mean_thresholds = np.array([])

if steane_thresholds.ndim == 2 and steane_thresholds.shape[1] == len(noise_levels):
    steane_mean_thresholds = np.mean(steane_thresholds, axis=0)
else:
    print("Issue with steane_thresholds dimensions")
    steane_mean_thresholds = np.array([])

if surface_thresholds.ndim == 2 and surface_thresholds.shape[1] == len(noise_levels):
    surface_mean_thresholds = np.mean(surface_thresholds, axis=0)
else:
    print("Issue with surface_thresholds dimensions")
    surface_mean_thresholds = np.array([])

if shor_fidelities.ndim == 2 and shor_fidelities.shape[1] == len(noise_levels):
    shor_mean_fidelities = np.mean(shor_fidelities, axis=0)
else:
    print("Issue with shor_fidelities dimensions")
    shor_mean_fidelities = np.array([])

if steane_fidelities.ndim == 2 and steane_fidelities.shape[1] == len(noise_levels):
    steane_mean_fidelities = np.mean(steane_fidelities, axis=0)
else:
    print("Issue with steane_fidelities dimensions")
    steane_mean_fidelities = np.array([])

if surface_fidelities.ndim == 2 and surface_fidelities.shape[1] == len(noise_levels):
    surface_mean_fidelities = np.mean(surface_fidelities, axis=0)
else:
    print("Issue with surface_fidelities dimensions")
    surface_mean_fidelities = np.array([])

# Ensure the arrays are not empty before plotting
if len(shor_mean_thresholds) > 0:
    plt.figure(figsize=(10, 6))
    plt.plot(noise_levels, shor_mean_thresholds, label="Shor Code Mean Error Rate", marker='o')
    plt.plot(noise_levels, steane_mean_thresholds, label="Steane Code Mean Error Rate", marker='o')
    plt.plot(noise_levels, surface_mean_thresholds, label="Surface Code Mean Error Rate", marker='o')
    plt.xlabel("Noise Probability")
    plt.ylabel("Mean Error Rate")
    plt.legend()
    plt.title("Mean Error Rate Comparison for Shor, Steane, and Surface Codes")
    plt.grid(True)
    plt.show()

if len(shor_mean_fidelities) > 0:
    plt.figure(figsize=(10, 6))
    plt.plot(noise_levels, shor_mean_fidelities, label="Shor Code Mean Fidelity", marker='o')
    plt.plot(noise_levels, steane_mean_fidelities, label="Steane Code Mean Fidelity", marker='o')
    plt.plot(noise_levels, surface_mean_fidelities, label="Surface Code Mean Fidelity", marker='o')
    plt.xlabel("Noise Probability")
    plt.ylabel("Mean Fidelity")
    plt.legend()
    plt.title("Mean Fidelity Comparison for Shor, Steane, and Surface Codes")
    plt.grid(True)
    plt.show()


Issue with shor_thresholds dimensions
Issue with steane_thresholds dimensions
Issue with surface_thresholds dimensions
Issue with shor_fidelities dimensions
Issue with steane_fidelities dimensions
Issue with surface_fidelities dimensions


In [87]:

def analyze_resources(backend, code_name):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit(include_measure=True)
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit(include_measure=True)
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit(include_measure=True)
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Use backend.target for transpilation
    qc_transpiled = transpile(
        qc_circuit,
        backend,
        target=backend.target
    )
    
    # Calculate resources
    depth = qc_transpiled.depth()
    gate_counts = qc_transpiled.count_ops()
    qubit_count = qc_transpiled.num_qubits
    
    # Calculate qubit overhead (physical qubits needed minus logical qubit)
    logical_qubit_count = 1  # Assuming we're encoding one logical qubit in these codes
    qubit_overhead = qubit_count - logical_qubit_count
    
    print(f"Code: {code_name}")
    print(f"Circuit Depth: {depth}")
    print(f"Gate Counts: {gate_counts}")
    print(f"Total Qubit Count: {qubit_count}")
    print(f"Qubit Overhead: {qubit_overhead}")
    
    return {
        'Code': code_name,
        'Circuit Depth': depth,
        'Gate Counts': gate_counts,
        'Total Qubit Count': qubit_count,
        'Qubit Overhead': qubit_overhead
    }

# Analyze resources and qubit overhead for each code
backend = FakeManhattanV2()
shor_resources = analyze_resources(backend, 'shor')
steane_resources = analyze_resources(backend, 'steane')
surface_resources = analyze_resources(backend, 'surface')

# Save resource usage results including qubit overhead
import csv
with open('code_resource_usage.csv', 'w', newline='') as csvfile:
    fieldnames = ['Code', 'Circuit Depth', 'Gate Counts', 'Total Qubit Count', 'Qubit Overhead']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerow(shor_resources)
    writer.writerow(steane_resources)
    writer.writerow(surface_resources)


Code: shor
Circuit Depth: 34
Gate Counts: OrderedDict({'cx': 43, 'rz': 14, 'measure': 9, 'sx': 7, 'barrier': 1})
Total Qubit Count: 65
Qubit Overhead: 64
Code: steane
Circuit Depth: 21
Gate Counts: OrderedDict({'cx': 15, 'rz': 8, 'measure': 7, 'sx': 4, 'barrier': 1})
Total Qubit Count: 65
Qubit Overhead: 64
Code: surface
Circuit Depth: 50
Gate Counts: OrderedDict({'cx': 42, 'rz': 30, 'sx': 15, 'measure': 13, 'barrier': 1})
Total Qubit Count: 65
Qubit Overhead: 64


In [88]:

from qiskit.result import Counts

def simulate_fault_tolerance(noise_level, backend, codes, repetitions=10):
    noise_models = {
        'Depolarizing': create_depolarizing_noise_model(noise_level),
        'Amplitude Damping': create_amplitude_damping_noise_model(noise_level),
        'Phase Damping': create_phase_damping_noise_model(noise_level),
        'Bit Flip': create_bit_flip_noise_model(noise_level),
        'Phase Flip': create_phase_flip_noise_model(noise_level),
        'Bit Phase Flip': create_bit_phase_flip_noise_model(noise_level),
        'Kraus': create_kraus_error_model(noise_level)
    }
    
    all_results = {}
    
    for code_name in codes:
        results = []
        
        for noise_model_name, noise_model in noise_models.items():
            for i in range(repetitions):
                if code_name == 'shor':
                    qc_circuit = shor_code_circuit(include_measure=True)
                elif code_name == 'steane':
                    qc_circuit = steane_code_circuit(include_measure=True)
                elif code_name == 'surface':
                    qc_circuit = surface_code_circuit(include_measure=True)
                else:
                    raise ValueError(f"Unknown code name: {code_name}")

                # Transpile the circuit with minimal optimization
                qc_transpiled = transpile(qc_circuit, backend, optimization_level=0)

                try:
                    # Run the noisy circuit
                    job = backend.run(qc_transpiled, noise_model=noise_model, shots=1024)
                    result = job.result()
                    counts_noisy = result.get_counts()

                    # Calculate error rate
                    correct_counts = counts_noisy.get('0' * qc_transpiled.num_clbits, 0)
                    error_rate = 1 - (correct_counts / 1024)

                    # You can use fidelity of measurement outcomes as a proxy if needed
                    # Ideally, you would compare distributions here if required
                    
                    results.append({
                        'Code': code_name,
                        'Noise Model': noise_model_name,
                        'Error Rate': error_rate,
                        'Repetition': i + 1
                    })

                    print(f"Code: {code_name}, Noise Model: {noise_model_name}, Error Rate: {error_rate}, Repetition: {i + 1}")

                except Exception as e:
                    print(f"Error in processing: {e}")
                    continue
        
        all_results[code_name] = results
    
    return all_results

# Usage
codes = ['shor', 'steane', 'surface']
backend = FakeManhattanV2()
noise_level = 0.05
fault_tolerance_results = simulate_fault_tolerance(noise_level=noise_level, backend=backend, codes=codes, repetitions=10)

# Save results for each code to separate CSV files
import csv
for code_name, results in fault_tolerance_results.items():
    csv_filename = f'{code_name}_fault_tolerance_results.csv'
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['Repetition', 'Code', 'Noise Model', 'Error Rate']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in results:
            writer.writerow(result)

    print(f"Results for {code_name} saved to {csv_filename}")


Code: shor, Noise Model: Depolarizing, Error Rate: 0.99609375, Repetition: 1
Code: shor, Noise Model: Depolarizing, Error Rate: 0.9970703125, Repetition: 2
Code: shor, Noise Model: Depolarizing, Error Rate: 0.9970703125, Repetition: 3
Code: shor, Noise Model: Depolarizing, Error Rate: 0.9990234375, Repetition: 4
Code: shor, Noise Model: Depolarizing, Error Rate: 0.998046875, Repetition: 5
Code: shor, Noise Model: Depolarizing, Error Rate: 0.998046875, Repetition: 6
Code: shor, Noise Model: Depolarizing, Error Rate: 1.0, Repetition: 7
Code: shor, Noise Model: Depolarizing, Error Rate: 0.998046875, Repetition: 8
Code: shor, Noise Model: Depolarizing, Error Rate: 0.9990234375, Repetition: 9
Code: shor, Noise Model: Depolarizing, Error Rate: 0.9970703125, Repetition: 10
Code: shor, Noise Model: Amplitude Damping, Error Rate: 1.0, Repetition: 1
Code: shor, Noise Model: Amplitude Damping, Error Rate: 1.0, Repetition: 2
Code: shor, Noise Model: Amplitude Damping, Error Rate: 1.0, Repetition: 

In [89]:
def evaluate_performance(backend, code_name):
    # Get noise models from your existing function
    noise_models = simulate_with_all_noise_models(0.05, backend)
    
    # Ensure noise_models is a list of dictionaries as returned by your simulate_with_all_noise_models function
    if not isinstance(noise_models, list):
        raise ValueError("The simulate_with_all_noise_models function should return a list.")

    if code_name == 'shor':
        qc_circuit = shor_code_circuit(include_measure=True)
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit(include_measure=True)
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit(include_measure=True)
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Start memory and time profiling
    tracemalloc.start()
    start_time = time.time()
    
    qc_transpiled = transpile(qc_circuit, backend)
    
    performance_results = []
    
    for result in noise_models:
        if result['Code'] == code_name:
            noise_model_name = result['Noise Model']
            fidelity = result['Fidelity']
            error_rate = result['Error Rate']

            end_time = time.time()
            execution_time = end_time - start_time

            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()

            print(f"Code: {code_name}, Noise Model: {noise_model_name}, Execution Time: {execution_time} seconds, Peak Memory Usage: {peak / 10**6} MB")
            
            performance_results.append({
                'Code': code_name,
                'Noise Model': noise_model_name,
                'Execution Time (Seconds)': execution_time,
                'Peak Memory Usage (MB)': peak / 10**6
            })
    
    return performance_results

# Performance analysis for Shor, Steane, and Surface codes on the current environment
for code_name in ['shor', 'steane', 'surface']:
    performance = evaluate_performance(backend, code_name)

    # Save performance results to a CSV file
    csv_filename = f'{code_name}_performance_comparison.csv'
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['Code', 'Noise Model', 'Execution Time (Seconds)', 'Peak Memory Usage (MB)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for result in performance:
            writer.writerow(result)

    print(f"Results for {code_name} saved to {csv_filename}")


Code: shor, Noise Model: Depolarizing, Fidelity: 0.001953125, Error Rate: 0.998046875
Code: shor, Noise Model: Amplitude Damping, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Phase Damping, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Bit Flip, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Phase Flip, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Bit Phase Flip, Fidelity: 0.0, Error Rate: 1.0
Code: shor, Noise Model: Kraus, Fidelity: 0.0, Error Rate: 1.0
Code: steane, Noise Model: Depolarizing, Fidelity: 0.01953125, Error Rate: 0.98046875
Code: steane, Noise Model: Amplitude Damping, Fidelity: 0.0615234375, Error Rate: 0.9384765625
Code: steane, Noise Model: Phase Damping, Fidelity: 0.060546875, Error Rate: 0.939453125
Code: steane, Noise Model: Bit Flip, Fidelity: 0.060546875, Error Rate: 0.939453125
Code: steane, Noise Model: Phase Flip, Fidelity: 0.052734375, Error Rate: 0.947265625
Code: steane, Noise Model: Bit Phase Flip, Fidelity: 0.0644

In [90]:
{
    "Depolarizing": create_depolarizing_noise_model,
    "Amplitude Damping": create_amplitude_damping_noise_model,
    # Add other noise models here
}


{'Depolarizing': <function __main__.create_depolarizing_noise_model(noise_level)>,
 'Amplitude Damping': <function __main__.create_amplitude_damping_noise_model(noise_level)>}

In [91]:
def analyze_resources(backend, code_name):
    if code_name == 'shor':
        qc_circuit = shor_code_circuit(include_measure=True)
    elif code_name == 'steane':
        qc_circuit = steane_code_circuit(include_measure=True)
    elif code_name == 'surface':
        qc_circuit = surface_code_circuit(include_measure=True)
    else:
        raise ValueError(f"Unknown code name: {code_name}")
    
    # Transpile the circuit for the backend
    qc_transpiled = transpile(qc_circuit, backend)
    
    # Calculate resources
    depth = qc_transpiled.depth()
    gate_counts = qc_transpiled.count_ops()
    qubit_count = qc_transpiled.num_qubits
    
    # Calculate qubit overhead (physical qubits needed minus logical qubit)
    logical_qubit_count = 1  # Assuming we're encoding one logical qubit in these codes
    qubit_overhead = qubit_count - logical_qubit_count
    
    print(f"Code: {code_name}")
    print(f"Circuit Depth: {depth}")
    print(f"Gate Counts: {gate_counts}")
    print(f"Total Qubit Count: {qubit_count}")
    print(f"Qubit Overhead: {qubit_overhead}")
    
    return {
        'Code': code_name,
        'Circuit Depth': depth,
        'Gate Counts': gate_counts,
        'Total Qubit Count': qubit_count,
        'Qubit Overhead': qubit_overhead
    }

# Analyze resources and qubit overhead for each code
for code_name in ['shor', 'steane', 'surface']:
    resources = analyze_resources(backend, code_name)
    
    # Save resource usage results to a CSV file
    csv_filename = f'{code_name}_resource_usage.csv'
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['Code', 'Circuit Depth', 'Gate Counts', 'Total Qubit Count', 'Qubit Overhead']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        writer.writerow(resources)

    print(f"Resource usage for {code_name} saved to {csv_filename}")


Code: shor
Circuit Depth: 39
Gate Counts: OrderedDict({'cx': 39, 'rz': 14, 'measure': 9, 'sx': 7, 'barrier': 1})
Total Qubit Count: 65
Qubit Overhead: 64
Resource usage for shor saved to shor_resource_usage.csv
Code: steane
Circuit Depth: 14
Gate Counts: OrderedDict({'cx': 12, 'rz': 8, 'measure': 7, 'sx': 4, 'barrier': 1})
Total Qubit Count: 65
Qubit Overhead: 64
Resource usage for steane saved to steane_resource_usage.csv
Code: surface
Circuit Depth: 55
Gate Counts: OrderedDict({'cx': 42, 'rz': 30, 'sx': 15, 'measure': 13, 'barrier': 1})
Total Qubit Count: 65
Qubit Overhead: 64
Resource usage for surface saved to surface_resource_usage.csv
